<a target="_blank" href="https://colab.research.google.com/github/Detroxsys/RP-2023-2/blob/main/Laboratorios/Lab05%20ISODATA%20implementacion/Lab05_Implementacion_ISODATA.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Laboratorio 05: Implementar ISODATA

In [2]:
#Librerías a utilizar
import numpy as np #Para distancias y manipulacion de arreglos
import matplotlib.pyplot as plt #Graficación.

## Implementa el algoritmo (7 puntos)

Implementar el algoritmo de ISODATA. 
* Comentar en el código que paso se esta implementando. 
* Documentar cada función utilizada. 

Lo siguiente es una sugerencia de la estructura, no es obligatorio seguirla al pie de la letra. 

In [101]:
class ISODATA(): 
  def __init__(self, data, centers): 
    """
    Función que recibe los parámetros del algorítmo donde
    data [np.ndarray]: los patrones a clasificar
    centers [np.ndarray]: los centros iniciales
    """
    self.m, self.n = data.shape # m = # de patrones n la dim.
    self.X = data.copy() # Para no modificar los originales
    self.c = len(centers) # la cantidad de centros iniciales
    self.Y = centers.copy() # Clusters iniciales
    self.Z = self.Y.copy() # Clusters iniciales

  def eculidean_dist(self, x, y):
    """
    Función que calcula distancia euclideana dados dos puntos
    """
    return np.linalg.norm(x-y)

  def assign_clusters(self): 
    """
    Función que asigna cada patrón a su cluster más cercano
    """
    self.classification = dict() # Para guardar la clasificación de cada patrón
    self.cluster_count = {cluster:0 for cluster in range(self.c)} # contamos elementos por cluster
    # Buscamos para cada patron
    for indx, patron in enumerate(self.X):
      # Obtenemos las distancias respecto a los centros
      distances = [self.eculidean_dist(patron, center) for center in self.Y]
      distances = np.array(distances)
      cluster = np.argmin(distances) # Encontramos la distancia mínima

      self.classification[indx] = cluster
      self.cluster_count[cluster] += 1

  def drop_clusters(self): 
    """
    Función que elimina un cluster si sus patrones
    asignados son menores a m0 y distribuye sus 
    patrones al resto de los clusters
    """
    # Verificamos los clusters que no cumplen la condición
    por_borrar = [center for center, count in self.cluster_count.items() if count < self.m0]
    # Los eliminamos
    self.Y = np.delete(self.Y, por_borrar, axis=0)
    self.c = len(self.Y) # actualizamos el número de centros
    self.assign_clusters() # Reasignamos clusters
    
  def update_centers(self): 
    """
    Actualizamos los centros de cada cluster
    """
    # sumamos los patrones de cada cluster y almacenamos en un arreglo
    Y_new = np.zeros_like(self.Y)
    for pat_indx, center in self.classification.items():
      Y_new[center] += self.X[pat_indx]
    
    Y_new /= np.array(
      tuple(self.cluster_count.values()) # Vector con el conteo de los patrones x cluster
      ).reshape(self.c,1) # reshape para que funcione np.divide entre vectores
    
    self.Y = Y_new.copy() # Actualizamos 

  def compute_distances(self): 
    """
    Función que calcula el promedio entre la distancia de cada cluster
    y sus respectivos patrones y la distancia global
    """
    d = np.zeros(self.c)
    # Sumamos la distancia de cada patron a su centro
    for pat_indx, center in self.classification.items():
      d[center] += self.eculidean_dist(self.X[pat_indx], self.Y[center])

    self.g_global = np.sum(d) / self.X.shape[0] # Para el paso 6
    
    # Promediamos por cluster
    d /= np.array(
      tuple(self.cluster_count.values()) # Vector con el conteo de los patrones x cluster
      )
    
    self.avg_dist = d # Paso 5


  def compute_std(self): 
    """
    Calcular la desviación estándar
    """
    std = np.zeros_like(self.Y)
    # Sumamos
    for pat_indx, center in self.classification.items():
      std[center] += np.power(self.X[pat_indx] - self.Y[center], 2)

    # Dividimos
    std /= np.array(tuple(self.cluster_count.values())).reshape(len(std), 1)
    # Sacamos raíz
    self.std = np.sqrt(std)

    # Guardamos los máximos de cada coordenada
    self.std_max = [(np.argmax(s), np.max(s)) for s in self.std]

  def splitting_clusters(self): 
    #Función para separar clusters. 
    pass
  def lumping_clusters(self): 
    #Función para combinar clusters. 
    pass
  def compute_dist_bwtn__centers(self): 
    #Calcular la distancia entre los centros 
    pass
  
  def classsifier(self, k, m0, s0, lamb, d0, l, eps, max_iter): 
    """
    Función que realiza isodata con los siguientes parámetros
    k: número de clusters deseados
    m0: mínimo de patrones para ser cluster
    s0: límite de desviación estándar para particiones
    lamb: cota para partición, lamb pertence (0,1)
    d0: cota para combinar clusters
    l: número máximo de clusters combinados simultáneamente
    eps: tolerancia
    max_iter: máximo de iteraciones
    """

    self.k = k
    self.m0 = m0
    self.s0 = s0
    self.lamb = lamb
    self.d0 = d0
    self.l = l
    self.eps = eps
    self.max_iter = max_iter

    # Paso 1
    # como S y L guardan el *historial* de las iteraciones
    # Necesitamos que sean vectores inicializados en 2
    self.it = 0 # Iteración actual
    self.S = np.zeros(self.max_iter+1) + 2
    self.L = self.S.copy()

    while self.it < self.max_iter+1:
      # Paso 2
      self.NC = 1 # Para indicar los cambios en los centros
      self.Z = self.Y.copy()
      self.assign_clusters() # asignamos los patrones a sus centros
      
      # Paso 3
      self.drop_clusters()

      # Paso 4
      self.update_centers()
      if len(self.Y) == self.c:
        suma_tol = np.sum(
          # arrglo con la distancia de los nuevos centros y los centros pasados
          [self.eculidean_dist(self.Y[indx], self.Z[indx]) for indx in range(self.c)] 
          ) # sumamos
        if suma_tol <= self.eps:
          self.NC = 0
      
      # Paso 5 y 6
      self.compute_distances()

      self.compute_std()

      # Paso 7
      if self.it == self.max_iter:
        # Paso 13
        pass
      else:
        if self.c <= (self.k + 1) / 2:
          # Paso 8
          self.S[self.it] = 0
          self.compute_std() # Calcular la desviación estándar

          # Paso 9
          if self.splitting_clusters(): # Si se dividieron clusters
            self.S[self.it] = 1
            continue
          else:
            if self.it > 1 and self.L[self.it-1] == 0: # Si no se partieron clusters en la pasada
              if self.NC == 0:
                break
              if self.NC == 1:
                continue
          

      self.it += 1




## Test del algoritmo (1 punto)

Prueba el algoritmo con los siguientes patrones y grafica el resultado. 

In [80]:
X = np.array([  [0,0], 
                [1,0], 
                [1,1], 
                [0,1],
                [4,1],
                [5,1], 
                [5,2],
                [7,1],
                [7,2],
                [6,3],
                [2,3],
                [1,4],
                [2,5],
                [3,5] ], dtype=float)

Y = np.array( [ [0,0], 
                [4,1],
                [7,2], 
                [2,3],
                [3,5]], dtype=float)

In [102]:
isodata = ISODATA(X, Y)
isodata.classsifier(k=3, m0=2, s0=1.5, lamb=0.5,
                    d0=2.5, l=2, max_iter=3, eps=0.1)

[[0.5        0.5       ]
 [0.47140452 0.47140452]
 [0.47140452 0.81649658]
 [0.5        0.5       ]
 [0.5        0.        ]]
[(0, 0.5), (0, 0.4714045207910317), (1, 0.816496580927726), (0, 0.5), (0, 0.5)]
[[0.5        0.5       ]
 [0.47140452 0.47140452]
 [0.47140452 0.81649658]
 [0.5        0.5       ]
 [0.5        0.        ]]
[(0, 0.5), (0, 0.4714045207910317), (1, 0.816496580927726), (0, 0.5), (0, 0.5)]
[[0.5        0.5       ]
 [0.47140452 0.47140452]
 [0.47140452 0.81649658]
 [0.5        0.5       ]
 [0.5        0.        ]]
[(0, 0.5), (0, 0.4714045207910317), (1, 0.816496580927726), (0, 0.5), (0, 0.5)]
[[0.5        0.5       ]
 [0.47140452 0.47140452]
 [0.47140452 0.81649658]
 [0.5        0.5       ]
 [0.5        0.        ]]
[(0, 0.5), (0, 0.4714045207910317), (1, 0.816496580927726), (0, 0.5), (0, 0.5)]


## Mini-Cuestionario (2 puntos)

### Opinión: 

### Conteste las 4 preguntas típicas de análisis y diseño de algoritmos


**1. ¿Es correcto?**

**2. ¿Complejidad temporal?**


**3. ¿Complejidad espacial?**


**4. ¿Ventajas y desventajas?**
